## Load libs

In [17]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score
import optuna

## Import data

In [18]:
insp = pd.read_csv("inspections.csv")
network = pd.read_csv("network.csv")
samplesub = pd.read_csv("sample_submission.csv")

## EDA

In [19]:
dataraw = insp.merge(network, how='inner', on='PipeId')

In [20]:
#filter a dataframe only where there is 1 in the column "Incidence"
df_filtered = dataraw.loc[dataraw['Incidence'] == 1] 

In [21]:
df_filtered["Material"].value_counts()

PE    8978
FD    1736
AO     993
CU     383
PN     111
ZD      90
PA      13
PV      12
FG       8
FO       4
Name: Material, dtype: int64

## Create our full features

In [22]:
df = pd.get_dummies(network['Province'])

# join the encoded dataframe with original dataframe
network = network.join(df)

df1 = pd.get_dummies(network['GasType'])

# join the encoded dataframe with original dataframe
network = network.join(df1)

df2 = pd.get_dummies(network['BoolBridle'])

# join the encoded dataframe with original dataframe
network = network.join(df2)

df3 = pd.get_dummies(network['Material'])

# join the encoded dataframe with original dataframe
network = network.join(df3)

network = network.drop(columns=['GasType', 'BoolBridle', 'Material',"Province","Town"])

In [23]:
data = insp.merge(network, how='inner', on='PipeId')

In [24]:
dataframe = data.drop(columns=["Incidence","InspectionDate"] ,axis=1)

In [25]:
dataframe["Volume"] = ((3.14*(dataframe["Diameter"]/2000))**2*dataframe["Length"])
dataframe

,PipeId,MaintenanceId,InspectionYear,MonthsLastRev,Severity,YearBuilt,Diameter,Length,Pressure,NumConnections,...,FD,FG,FI,FO,PA,PE,PN,PV,ZD,Volume
0,189311802,ZRV-00001972,2010,24,4,2001,160.0,117.831,4.000,0,...,0,0,0,0,0,1,0,0,0,7.435306
1,189311802,ZRV-00001972,2012,24,4,2001,160.0,117.831,4.000,0,...,0,0,0,0,0,1,0,0,0,7.435306
2,189311802,ZRV-00001972,2014,24,4,2001,160.0,117.831,4.000,0,...,0,0,0,0,0,1,0,0,0,7.435306
3,189311802,ZRV-00001972,2016,24,4,2001,160.0,117.831,4.000,0,...,0,0,0,0,0,1,0,0,0,7.435306
4,189311802,ZRV-00001972,2018,24,4,2001,160.0,117.831,4.000,0,...,0,0,0,0,0,1,0,0,0,7.435306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,ZRV-00004581,2020,24,4,1993,63.0,1.237,4.000,0,...,0,0,0,0,0,1,0,0,0,0.012102
6345340,56907425,ZRV-00004581,2020,24,4,2000,160.0,169.732,4.000,1,...,0,0,0,0,0,1,0,0,0,10.710334
6345341,7843615,ZRV-00002112,2021,24,4,1995,150.0,0.361,0.025,0,...,1,0,0,0,0,0,0,0,0,0.020021
6345342,56922456,ZRV-00004581,2020,24,4,1993,63.0,23.306,4.000,0,...,0,0,0,0,0,1,0,0,0,0.228007


In [26]:
dataframe = dataframe.drop(columns=["MaintenanceId"], axis=1)
dataframe

,PipeId,InspectionYear,MonthsLastRev,Severity,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,...,FD,FG,FI,FO,PA,PE,PN,PV,ZD,Volume
0,189311802,2010,24,4,2001,160.0,117.831,4.000,0,0,...,0,0,0,0,0,1,0,0,0,7.435306
1,189311802,2012,24,4,2001,160.0,117.831,4.000,0,0,...,0,0,0,0,0,1,0,0,0,7.435306
2,189311802,2014,24,4,2001,160.0,117.831,4.000,0,0,...,0,0,0,0,0,1,0,0,0,7.435306
3,189311802,2016,24,4,2001,160.0,117.831,4.000,0,0,...,0,0,0,0,0,1,0,0,0,7.435306
4,189311802,2018,24,4,2001,160.0,117.831,4.000,0,0,...,0,0,0,0,0,1,0,0,0,7.435306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,2020,24,4,1993,63.0,1.237,4.000,0,0,...,0,0,0,0,0,1,0,0,0,0.012102
6345340,56907425,2020,24,4,2000,160.0,169.732,4.000,1,0,...,0,0,0,0,0,1,0,0,0,10.710334
6345341,7843615,2021,24,4,1995,150.0,0.361,0.025,0,0,...,1,0,0,0,0,0,0,0,0,0.020021
6345342,56922456,2020,24,4,1993,63.0,23.306,4.000,0,0,...,0,0,0,0,0,1,0,0,0,0.228007


In [ ]:
dataframe.columns

In [ ]:
dataframe

## Create the data

In [ ]:
samplesub

,PipeId,Incidence
0,446859944,0
1,428124500,0
2,438428871,0
3,429034569,0
4,411184477,0
...,...,...
909728,235426649,0
909729,235426673,0
909730,235426707,0
909731,190956601,0


In [31]:
samplesub = samplesub.drop(columns="Incidence",axis=1)
finalsub = samplesub.merge(dataframe, how='left', on='PipeId')
finalsub = finalsub.drop(columns="PipeId")


In [34]:
finalsub

,InspectionYear,MonthsLastRev,Severity,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,Albacete,...,FD,FG,FI,FO,PA,PE,PN,PV,ZD,Volume
0,2013.0,12.0,4.0,2012.0,110.0,44.286,0.15,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.320843
1,2015.0,24.0,4.0,2012.0,110.0,44.286,0.15,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.320843
2,2017.0,24.0,4.0,2012.0,110.0,44.286,0.15,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.320843
3,2019.0,23.0,4.0,2012.0,110.0,44.286,0.15,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.320843
4,2013.0,12.0,4.0,2012.0,160.0,110.715,0.15,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.986276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995627,2012.0,24.0,4.0,2004.0,160.0,2.325,4.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.146711
3995628,2014.0,24.0,4.0,2004.0,160.0,2.325,4.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.146711
3995629,2016.0,23.0,4.0,2004.0,160.0,2.325,4.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.146711
3995630,2018.0,25.0,4.0,2004.0,160.0,2.325,4.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.146711


## SMOTE

In [ ]:
# from imblearn.over_sampling import SMOTE

# X = dataframe.drop('Incidence', axis=1)
# y = dataframe['Incidence']

# # Instantiate SMOTE
# smote = SMOTE()

# # Apply SMOTE to your dataset
# X_resampled, y_resampled = smote.fit_resample(X, y)


# data_resampledful = X_resampled.merge(y_resampled, left_index=True,right_index=True)


# data_resampledful = data_resampledful.drop(columns="PipeId", axis=1)
# data_resampledful

### Undersampling

In [ ]:
df = dataframe.sample(frac=1)
incidence_df = dataframe.loc[dataframe['Incidence'] == 1]
non_incidence_df = dataframe.loc[dataframe['Incidence'] == 0][:3200]
normal_distributed_df = pd.concat([incidence_df, non_incidence_df])
new_df = normal_distributed_df.sample(frac=1, random_state=42)
new_df = new_df.drop(columns="PipeId",axis=1)
new_df

# Hypertune the model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_df.loc[:, new_df.columns != "Incidence"], new_df["Incidence"], test_size=0.2, random_state=42)

In [ ]:
#USING OPTUNA TO GET THE BEST PARAMS
def objective(trial):
    # Define the hyperparameters to search over
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000, step=50),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
    }
    
    # Train an XGBoost classifier with the specified hyperparameters
    clf = xgb.XGBClassifier(**params)
    clf.fit(X_train, y_train)
    
    # Calculate the auc on the test set
    y_pred = clf.predict(X_test)
    auc_res = roc_auc_score(y_test, y_pred)
    
    # Return the accuracy as the objective value for Optuna to optimize
    return auc_res

# Use Optuna to search for the best hyperparameters
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Print the best hyperparameters found by Optuna
print(f'Best parameters: {study.best_params}')
print(f'Best auc: {study.best_value:.3f}')

In [ ]:
from xgboost import XGBClassifier
X_full = pd.concat([X_train,X_test])
y_full = pd.concat([y_train,y_test])

In [ ]:
my_model = XGBClassifier(n_estimators = 300, objective='binary:logistic',max_depth = 10, learning_rate= 0.07872370673469276, subsample= 0.8042727399203574, colsample_bytree= 0.8331595047059349, gamma= 2.348057633779575, min_child_weight= 8)
my_model.fit(X_full, y_full)

## Predict the kaggle results

In [ ]:
results_1 = my_model.predict(finalsub)
results1df = pd.DataFrame(results_1)
results1df = pd.DataFrame(results_1)


finalfinal = samplesub_sub.merge(results1df, left_index = True,right_index=True)
finalfinal.rename(columns={0: "Incidence"}, inplace=True)
finalfinal

## Create the submission file

In [ ]:
#finalfinal.to_csv("SMOTEOPTUNAfeatures2M.csv", index=False)